# Deploy cohere-rerank-v3.5 Model Package from AWS Marketplace 


Cohere builds a collection of Large Language Models (LLMs) trained on a massive corpus of curated web data. Powering these models, our infrastructure enables our product to be deployed for a wide range of use cases. The use cases we power include generation (copy writing, etc), summarization, classification, content moderation, information extraction, semantic search, and contextual entity extraction

This sample notebook shows you how to deploy [cohere-rerank-v3.5](https://aws.amazon.com/marketplace/pp/prodview-nhyphjamrbx36) using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [cohere-rerank-v3.5](https://aws.amazon.com/marketplace/pp/prodview-nhyphjamrbx36). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Delete the endpoint](#H.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [cohere-rerank-v3.5](https://aws.amazon.com/marketplace/pp/prodview-nhyphjamrbx36)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
!pip install --upgrade setuptools==69.5.1 cohere-aws zipp cohere
# if you upgrade the package, you need to restart the kernel

from cohere_aws import Client
import boto3

In [ ]:
# To-Do update package once sagemaker publishes
cohere_package = "cohere-rerank-v3-5-v1-1-011720-494e623cd6db335598198b1855d448a8"

model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{cohere_package}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{cohere_package}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{cohere_package}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{cohere_package}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{cohere_package}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{cohere_package}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{cohere_package}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{cohere_package}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{cohere_package}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{cohere_package}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{cohere_package}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{cohere_package}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{cohere_package}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{cohere_package}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{cohere_package}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{cohere_package}",
}

region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise Exception("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Create an endpoint

In [ ]:
co = Client(region_name=region)
endpoint_name="cohere-rerank-v3-5"
co.create_endpoint(arn=model_package_arn, endpoint_name=endpoint_name, instance_type="ml.g5.xlarge", n_instances=1)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [ ]:
documents = [
    'Title: Incorrect Password\nContent: Hello, I have been trying to access my account for the past hour and it keeps\n  saying my password is incorrect. Can you please help me?\n',
    'Title: Confirmation Email Missed\nContent: Hi, I recently purchased a product from your website but I never received\n  a confirmation email. Can you please look into this for me?\n',
    'Title: Questions about Return Policy\nContent: Hello, I have a question about the return policy for this product. I purchased\n  it a few weeks ago and it is defective.\n',
    'Title: Customer Support is Busy\nContent: Good morning, I have been trying to reach your customer support team for\n  the past week but I keep getting a busy signal. Can you please help me?\n',
    'Title: Received Wrong Item\nContent: Hi, I have a question about my recent order. I received the wrong item and\n  I need to return it.\n',
    'Title: Customer Service is Unavailable\nContent: Hello, I have been trying to reach your customer support team for the past\n  hour but I keep getting a busy signal. Can you please help me?\n',
    'Title: Return Policy for Defective Product\nContent: Hi, I have a question about the return policy for this product. I purchased\n  it a few weeks ago and it is defective.\n',
    'Title: Wrong Item Received\nContent: Good morning, I have a question about my recent order. I received the wrong\n  item and I need to return it.\n',
    'Title: Return Defective Product\nContent: Hello, I have a question about the return policy for this product. I purchased\n  it a few weeks ago and it is defective.\n'
    ]

### C. Perform real-time inference

In [ ]:
import cohere

co_sdk = cohere.SagemakerClientV2()

response = co_sdk.rerank(model=endpoint_name, documents=documents, query='What emails have been about returning items?', top_n=5)

### D. Visualize output

In [ ]:
print(f'Documents: {response}') 

## 4. Clean-up

### A. Delete the model

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
co.delete_endpoint()
co.close()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

